<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Hello,-world!" data-toc-modified-id="Hello,-world!-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Hello, world!</a></span><ul class="toc-item"><li><span><a href="#Main-entry" data-toc-modified-id="Main-entry-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Main entry</a></span></li><li><span><a href="#GetLine" data-toc-modified-id="GetLine-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>GetLine</a></span></li><li><span><a href="#IO-monad" data-toc-modified-id="IO-monad-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>IO monad</a></span></li><li><span><a href="#Example-of-IO" data-toc-modified-id="Example-of-IO-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Example of IO</a></span></li><li><span><a href="#Return" data-toc-modified-id="Return-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Return</a></span></li><li><span><a href="#Useful-I/O-functions" data-toc-modified-id="Useful-I/O-functions-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Useful I/O functions</a></span><ul class="toc-item"><li><span><a href="#putStr" data-toc-modified-id="putStr-1.6.1"><span class="toc-item-num">1.6.1&nbsp;&nbsp;</span>putStr</a></span></li><li><span><a href="#putChar" data-toc-modified-id="putChar-1.6.2"><span class="toc-item-num">1.6.2&nbsp;&nbsp;</span>putChar</a></span></li><li><span><a href="#print" data-toc-modified-id="print-1.6.3"><span class="toc-item-num">1.6.3&nbsp;&nbsp;</span>print</a></span></li><li><span><a href="#putStrLn-vs-print" data-toc-modified-id="putStrLn-vs-print-1.6.4"><span class="toc-item-num">1.6.4&nbsp;&nbsp;</span>putStrLn vs print</a></span></li><li><span><a href="#getChar" data-toc-modified-id="getChar-1.6.5"><span class="toc-item-num">1.6.5&nbsp;&nbsp;</span>getChar</a></span></li><li><span><a href="#When" data-toc-modified-id="When-1.6.6"><span class="toc-item-num">1.6.6&nbsp;&nbsp;</span>When</a></span></li><li><span><a href="#sequence" data-toc-modified-id="sequence-1.6.7"><span class="toc-item-num">1.6.7&nbsp;&nbsp;</span>sequence</a></span></li><li><span><a href="#sequence-with" data-toc-modified-id="sequence-with-1.6.8"><span class="toc-item-num">1.6.8&nbsp;&nbsp;</span>sequence with</a></span></li><li><span><a href="#What's-with-the-[(),(),(),(),()]-at-the-end?" data-toc-modified-id="What's-with-the-[(),(),(),(),()]-at-the-end?-1.6.9"><span class="toc-item-num">1.6.9&nbsp;&nbsp;</span>What's with the <code>[(),(),(),(),()]</code> at the end?</a></span></li><li><span><a href="#mapM" data-toc-modified-id="mapM-1.6.10"><span class="toc-item-num">1.6.10&nbsp;&nbsp;</span>mapM</a></span></li><li><span><a href="#forever" data-toc-modified-id="forever-1.6.11"><span class="toc-item-num">1.6.11&nbsp;&nbsp;</span>forever</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-1.6.12"><span class="toc-item-num">1.6.12&nbsp;&nbsp;</span>Summary</a></span></li></ul></li><li><span><a href="#Do-is-not-as-simple-as-you-thought!" data-toc-modified-id="Do-is-not-as-simple-as-you-thought!-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Do is not as simple as you thought!</a></span></li></ul></li><li><span><a href="#Other-I/O-functions" data-toc-modified-id="Other-I/O-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Other I/O functions</a></span></li></ul></div>

Input and Output
================

<img src="img/dognap.png" title="poor dog" style="float:right;margin-left:2em;" />

* Haskell is a purely functional language. 
* In imperative languages you usually get things done by giving the computer
a series of steps to execute, functional programming is more of defining
what stuff is. 
* In Haskell, a function can't change some state, like
changing the contents of a variable (when a function changes state, we
say that the function has *side-effects*). 
* The only thing a function can
do in Haskell is give us back some result based on the parameters we
gave it. 
* If a function is called two times with the same parameters, it
has to return the same result. 

* If a function can't change anything in the world, how is it supposed to tell us what
it calculated? 

* Haskell has
a really clever (weird) system for dealing with functions that have side-effects
that neatly separates the part of our program that is pure and the part
of our program that is impure, which does all the dirty work like
talking to the keyboard and the screen. 
* With those two parts separated,
we can still reason about our pure program and take advantage of all the
things that purity offers, like laziness, robustness and modularity
while efficiently communicating with the outside world.

> __Jupyter Note:__ We'll turn off the [automatic linting for IHaskell](https://github.com/gibiansky/IHaskell/wiki#opt-no-lint) first.

In [1]:
:opt no-lint

> __Jupyter Note:__ About the following functions.
>
> 

* `withStdin` takes a string and runs an [`IO`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:IO) action with that string as the standard input.
This is for simulating an interactive command line environment for programs. 

* `catchPrint` catches an exception and prints it to standard output. We use this because Jupyter won't show us the output of a program if the program throws an exception. We want to see both the output and the exception.

In [2]:
import System.IO
import GHC.IO.Handle
import Control.Exception
import System.Directory

withStdin :: String -> IO a -> IO a
withStdin s action = do
    writeFile "stdin.txt" s
    finally
        (bracket 
            (openFile "stdin.txt" ReadWriteMode)
            hClose
            (\h -> do
                  stdin' <- hDuplicate stdin
                  hDuplicateTo h stdin
                  finally action (hDuplicateTo stdin' stdin)            
            )
        )
        (removeFile "stdin.txt")

catchPrint = flip catch p where
    p :: SomeException -> IO ()
    p = print

## Hello, world!


<img src="img/helloworld.png" title="HELLO!" style="float:left;margin-right:2em;" />

* We've always loaded our functions into GHCI to test them
out and play with them. 
* We've also explored the standard library
functions that way. 
* We're finally going to write our first *real* Haskell program! 

In [3]:
main = putStrLn "hello, world"

In [4]:
main

hello, world

We just defined a name called `main` and in it we call a function called
[`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn) with the parameter `"hello, world"`. 

Let's examine what we wrote. First, let's look at the type of the
function [`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn).

In [5]:
:t putStrLn

putStrLn :: String -> IO ()

In [6]:
:t putStrLn "hello, world"

putStrLn "hello, world" :: IO ()

* We can read the type of [`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn) like this: [`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn) takes a string and
returns an *I/O action* that has a result type of `()` (i.e. the empty
tuple, also know as unit). 

* An I/O action is something that, when
performed, will carry out an action with a side-effect (that's usually
either reading from the input or printing stuff to the screen) and will
also contain some kind of return value inside it. 

* Printing a string to
the terminal doesn't really have any kind of meaningful return value, so
a dummy value of `()` is used.

> The empty tuple is a value of `()` and it also has a type of `()`.

### Main entry

* An I/O action will be performed when we give it a name of `main` and
then run our program.

* We can use *do* syntax to glue together several I/O actions
into one. 

In [7]:
main = do
    putStrLn "Hello, what's your name?"
    name <- getLine
    putStrLn ("Hey " ++ name ++ ", you rock!")

* This part is  an imperative program. 
* Notice that we said *do* and then we
laid out a series of steps, like we would in an imperative program. Each
of these steps is an I/O action. By putting them together with *do*
syntax, we glued them into one I/O action. 

* The action that we got has a
type of `IO ()`, because that's the type of the last I/O action inside.

* By convention, we
don't usually specify a type declaration for `main`.


In [8]:
:t getLine

getLine :: IO String

### GetLine

<img src="img/luggage.png" title="luggage" style="float:right;margin-right:2em;" />

*  __[`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine)__ is an I/O action that contains a result type of
[`String`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:String). It will wait for the user to input
something at the terminal and then that something will be represented as
a string. 

* `name <- getLine`: *perform the I/O action [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) and then bind
its result value to `name`*. 

* [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) has a type of `IO String`, so `name` will
have a type of [`String`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:String). 



### IO monad

* You can think of an I/O action as a box with
little feet that will go out into the real world and do something there
and maybe bring back some data.

* Once it's fetched that data for you, the only way to open the box and
get the data inside it is to use the `<-` construct. 

* If we're taking
data out of an I/O action, we can only take it out when we're inside
another I/O action. 

* This is how Haskell manages to neatly separate the
pure and impure parts of our code. Weird!


* [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) is in a sense impure because its result value is not guaranteed to be the same when performed twice.

* That's why it's sort of *tainted* with the [`IO`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:IO) type constructor and we
can only get that data out in I/O code. 

* When we do `name <- getLine`, `name` is just a normal string,
because it represents what's inside the box. We can have a really
complicated function that, say, takes your name (a normal string) as a
parameter and tells you your fortune and your whole life's future based
on your name. We can do this:

In [9]:
main = do
    putStrLn "Hello, what's your name?"
    name <- getLine
    putStrLn $ "Read this carefully, because this is your future: " ++ tellFortune name

: 

Take a look at this piece of code. Is it valid?

In [10]:
nameTag = "Hello, my name is " ++ getLine

: 

* We first have to get the result out of the I/O action to get a
value of type [`String`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:String) and the only way to do that is to say something
like `name <- getLine` inside some other I/O action. 

* If we want to deal
with impure data, we have to do it in an impure environment. So the
taint of impurity spreads around much like the undead scourge and it's
in our best interest to keep the I/O parts of our code as small as
possible.

Every I/O action that gets performed has a result encapsulated within
it. That's why our previous example program could also have been written
like this:

In [11]:
main = do
    foo <- putStrLn "Hello, what's your name?"
    name <- getLine
    putStrLn ("Hey " ++ name ++ ", you rock!")

* `foo` would just have a value of `()`. 

* Notice that we didn't bind the last [`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn) to anything.
That's because in a *do* block, *the last action cannot be bound to a
name* like the first two were. 
* the *do* block automatically extracts the
value from the last action and binds it to its own result.
* Except for the last line, every line in a *do* block that doesn't bind
can also be written with a bind. 

How about this one?

In [12]:
name = getLine


* I/O actions will only be performed when they are given a name of `main` or
when they're inside a bigger I/O action that we composed with a *do*
block. 
* We can also use a *do* block to glue together a few I/O actions
and then we can use that I/O action in another *do* block and so on.
Either way, they'll be performed only if they eventually fall into `main`.


In [13]:
putStrLn "HEEY"

HEEY

In [14]:
import Data.Char

main = do
    putStrLn "What's your first name?"
    firstName <- getLine
    putStrLn "What's your last name?"
    lastName <- getLine
    let bigFirstName = map toUpper firstName
        bigLastName = map toUpper lastName
    putStrLn $ "hey " ++ bigFirstName ++ " " ++ bigLastName ++ ", how are you?"

In [15]:
withStdin (unlines ["John", "Brown"]) main

What's your first name?
What's your last name?
hey JOHN BROWN, how are you?

 Had we done something like
`let firstName = getLine`, we would have just called the [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) I/O action a different
name and we'd still have to run it through a `<-` to perform it.

### Example of IO
Now we're going to make a program that continuously reads a line and
prints out the same line with the words reversed. The program's
execution will stop when we input a blank line. This is the program:

In [16]:
main = do
    line <- getLine
    if null line
        then return ()
        else do
            putStrLn $ reverseWords line
            main

reverseWords :: String -> String
reverseWords = unwords . map reverse . words
-- `reverseWords` function is just a
-- normal function that takes a string like "hey there man" and then calls
-- [`words`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:words) with it to produce a list of words like `["hey","there","man"]`.
-- Then we map [`reverse`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:reverse) on the list, getting `["yeh","ereht","nam"]` and then
-- we put that back into one string by using [`unwords`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:unwords) and the final result
-- is `"yeh ereht nam"`. See how we used function composition here. Without
-- function composition, we'd have to write something like
-- `reverseWords st = unwords (map reverse (words st))`.

To get a feel of what it does, you can run it before we go over the
code.

In [17]:
withStdin "hey there man" $ catchPrint main

yeh ereht nam
<stdin>: hGetLine: end of file

### Return

*  The [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) in Haskell is really nothing like the [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) in most
other languages! 
* In imperative languages, [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return)
usually ends the execution of a method or subroutine and makes it report
some sort of value to whoever called it. 
* In Haskell (in I/O actions
specifically), it makes an I/O action out of a pure value. 
* If you think
about the box analogy from before, it takes a value and wraps it up in a
box. The resulting I/O action doesn't actually do anything, it just has
that value encapsulated as its result. 
* So in an I/O context,
`return "haha"` will have a type of `IO String`. 
* Using [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) doesn't cause the I/O *do* block to end in execution or
anything like that. 

In [18]:
main = do
    return ()
    return "HAHAHA"
    line <- getLine
    return "BLAH BLAH BLAH"
    return 4
    putStrLn line

* We can use [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) in combination
with `<-` to bind stuff to names.

In [19]:
main = do
    a <- return "hell"
    b <- return "yeah!"
    putStrLn $ a ++ " " ++ b

* So you see, [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) is sort of the opposite to `<-`. 
* While [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) takes a
value and wraps it up in a box, `<-` takes a box (and performs it) and
takes the value out of it, binding it to a name. 
* But doing this is kind
of redundant, especially since you can use *let* bindings in *do* blocks
to bind to names, like so:

In [20]:
main = do
    let a = "hell"
        b = "yeah"
    putStrLn $ a ++ " " ++ b

> A *do* block can also have just one I/O action. In that case, it's the
> same as just writing the I/O action. Some people would prefer writing
> `then do return ()` in this case because the *else* also has a *do*.

### Useful I/O functions

#### putStr

__[`putStr`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStr)__ is much like [`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn) in that it takes a string as a parameter
and returns an I/O action that will print that string to the terminal,
only [`putStr`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStr) doesn't jump into a new line after printing out the string
while [`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn) does.

In [21]:
main = do   putStr "Hey, "
            putStr "I'm "
            putStrLn "Andy!"

In [22]:
main

Hey, I'm Andy!

Its type signature is `putStr :: String -> IO ()`, so the result
encapsulated within the resulting I/O action is the unit. A dud value,
so it doesn't make sense to bind it.

#### putChar
__[`putChar`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putChar)__ takes a character and returns an I/O action that will print it
out to the terminal.

In [23]:
main = do   putChar 't'
            putChar 'e'
            putChar 'h'

In [24]:
main

teh

* [`putStr`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStr) is actually defined recursively with the help of [`putChar`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putChar). 

* The
edge condition of [`putStr`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStr) is the empty string, so if we're printing an
empty string, just return an I/O action that does nothing by using
`return ()`. If it's not empty, then print the first character of the
string by doing [`putChar`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putChar) and then print of them using [`putStr`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStr).

In [25]:
putStr :: String -> IO ()
putStr [] = return ()
putStr (x:xs) = do
    putChar x
    putStr xs

#### print

__[`print`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:print)__ takes a value of any type that's an instance of [`Show`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Show) (meaning that
we know how to represent it as a string), calls [`show`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:show) with that value to
stringify it and then outputs that string to the terminal. Basically,
it's just `putStrLn . show`. It first runs [`show`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:show) on a value and then feeds
that to [`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn), which returns an I/O action that will print out our
value.

In [26]:
main = do   print True
            print 2
            print "haha"
            print 3.2
            print [3,4,3]

In [27]:
main

True
2
"haha"
3.2
[3,4,3]

GHCI actually uses [`print`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:print) on that
value to display it on our terminal!

In [28]:
3

3

In [29]:
print 3

3

In [30]:
map (++"!") ["hey","ho","woo"]

["hey!","ho!","woo!"]

In [31]:
print (map (++"!") ["hey","ho","woo"])

["hey!","ho!","woo!"]

#### putStrLn vs print
When we want to print out strings, we usually use [`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn) because we
don't want the quotes around them, but for printing out values of other
types to the terminal, [`print`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:print) is used the most.

#### getChar

__[`getChar`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getChar)__ is an I/O action that reads a character from the input. Thus,
its type signature is `getChar :: IO Char`, because the result contained
within the I/O action is a [`Char`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Char). Note that due to buffering, reading of
the characters won't actually happen until the user mashes the return
key.

In [32]:
main = do
    c <- getChar
    if c /= ' '
        then do
            putChar c
            main
        else return ()

This program looks like it should read a character and then check if
it's a space. If it is, halt execution and if it isn't, print it to the
terminal and then do the same thing all over again. Well, it kind of
does, only not in the way you might expect. Check this out:

In [33]:
withStdin "hello sir" main

hello

#### When

* The __[`when`](https://hackage.haskell.org/package/base/docs/Control-Monad.html#v:when)__ function is found in `Control.Monad` (to get access to it, do
`import Control.Monad`). 
* It's interesting because in a *do* block it looks
like a control flow statement, but it's actually a normal function. 
* It takes a boolean value and an I/O action if that boolean value is [`True`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:True),
it returns the same I/O action that we supplied to it. 
* However, if it's
[`False`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:False), it returns the `return ()`, action, so an I/O action that doesn't
do anything. 

By using [`when`](https://hackage.haskell.org/package/base/docs/Control-Monad.html#v:when):

In [34]:
import Control.Monad

main = do
    c <- getChar
    when (c /= ' ') $ do
        putChar c
        main

In [35]:
withStdin "hello sir" main

hello

#### sequence

* __[`sequence`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:sequence)__ takes a list of I/O actions and returns an I/O action that
will perform those actions one after the other. 
* The result contained in
that I/O action will be a list of the results of all the I/O actions
that were performed. 
* Its type signature is
`sequence :: [IO a] -> IO [a]`. Doing this:

In [36]:
main = do
    a <- getLine
    b <- getLine
    c <- getLine
    print [a,b,c]

Is exactly the same as doing this:.

In [37]:
main = do
    rs <- sequence [getLine, getLine, getLine]
    print rs

So `sequence [getLine, getLine, getLine]` makes an I/O action that will
perform [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) three times. If we bind that action to a name, the
result is a list of all the results, so in our case, a list of three
things that the user entered at the prompt.

#### sequence with 
A common pattern with [`sequence`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:sequence) is when we map functions like [`print`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:print) or
[`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn) over lists. Doing `map print [1,2,3,4]` won't create an I/O
action. It will create a list of I/O actions, because that's like
writing `[print 1, print 2, print 3, print 4]`. If we want to transform
that list of I/O actions into an I/O action, we have to sequence it.

In [38]:
sequence (map print [1,2,3,4,5])

1
2
3
4
5
[(),(),(),(),()]

#### What's with the `[(),(),(),(),()]` at the end? 

#When we evaluate an
I/O action in GHCI, it's performed and then its result is printed out,
unless that result is `()`, in which case it's not printed out. 
* That's why
evaluating `putStrLn "hehe"` in GHCI just prints out `hehe` (because the
contained result in `putStrLn "hehe"` is `()`). 
* But when we do [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) in
GHCI, the result of that I/O action is printed out, because [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) has
a type of `IO String`.

#### mapM

* Because mapping a function that returns an I/O action over a list and
then sequencing it is so common, the utility functions __[`mapM`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:mapM)__ and __`mapM_`__
were introduced. 
* [`mapM`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:mapM) takes a function and a list, maps the function
over the list and then sequences it. 
* `mapM_` does the same, only it
throws away the result later. We usually use `mapM_` when we don't care
what result our sequenced I/O actions have.

In [39]:
mapM print [1,2,3]

1
2
3
[(),(),()]

In [40]:
mapM_ print [1,2,3]

1
2
3

#### forever

__[`forever`](https://hackage.haskell.org/package/base/docs/Control-Monad.html#v:forever)__ takes an I/O action and returns an I/O action that just repeats
the I/O action it got forever. It's located in `Control.Monad`. 


This
little program will indefinitely ask the user for some input and spit it
back to him, CAPSLOCKED:

In [41]:
import Control.Monad
import Data.Char

main = forever $ do
    putStr "Give me some input: "
    l <- getLine
    putStrLn $ map toUpper l

In [42]:
withStdin "capslock" $ catchPrint main

Give me some input: CAPSLOCK
Give me some input: <stdin>: hGetLine: end of file

#### Summary

* We learned the basics of input and output. We also
found out what I/O actions are, how they enable us to do input and
output and when they are actually performed. 
* I/O actions
are values much like any other value in Haskell. We can pass them as
parameters to functions and functions can return I/O actions as results.
* What's special about them is that if they fall into the `main` function
(or are the result in a GHCI line), they are performed. 

* Don't think of a function like [`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn) as a function that takes a
string and prints it to the screen. Think of it as a function that takes
a string and returns an I/O action (same story). That I/O action will, when
performed, print beautiful poetry to your terminal.

### Do is not as simple as you thought!

* In Haskell, both >>= and >> are functions from the Monad class. This means they're overloaded differently for every monad: (>>=)::m a->(a->m b)->m b
* You start with an m a, apply the a -> m b function to it to get a m (m b) (this is the
map part) and then you flatten the m (m b) into an m b (this is the flat part).
* List monad

In [43]:
[1,2,3,4] >>= \x->[x-1,x+1]  -- WHAT?!

[0,2,1,3,2,4,3,5]

In [44]:
do {
    a <- [1,2,3];
    return a;
    }

[1,2,3]

In [45]:
[1,2,3] >>= \x -> [x]

[1,2,3]

## Other I/O functions

* Files and streams
* Command line arguments
* Randomness
* Bytestrings
* Exceptions